In [1]:
from typing import List, Dict, Iterable, Tuple, Callable
from matplotlib import pyplot as plt
from collections import Counter
import sys
import os
import string
import urllib.request
import requests
import curl
import pycurl
import webbrowser
import numpy as np
import math
import pandas as pd
# from IPython import qt
from matplotlib.pyplot import figure
from py.xml import raw
from requests.api import get
from matplotlib import pyplot as plt
# from scratch.working_with_data import rescale
# from scratch.multiple_regression import least_squares_fit, predict
# from scratch.gradient_descent import gradient_step

# from stats import mean, median, de_mean, standard_deviation, correlation
# from gradient_descent import minimize_stochastic, maximize_stochastic, maximize_batch
# from vector import dot, vector_add
# from normal import normal_cdf
# from matrix import make_matrix, get_column, shape, matrix_multiply
# from logistic_regression import *

import math
import os
import random
import sys
from functools import partial, reduce

from scipy.optimize import fmin_tnc

import tqdm

from typing import*

from collections import*


In [2]:
# Imports dataset
df = pd.read_csv('Flight Delays Data.csv')
df.head()

,Year,Month,DayofMonth,DayOfWeek,Carrier,OriginAirportID,DestAirportID,CRSDepTime,DepDelay,DepDel15,CRSArrTime,ArrDelay,ArrDel15,Cancelled
0,2013,4,19,5,DL,11433,13303,837,-3.0,0.0,1138,1.0,0.0,0.0
1,2013,4,19,5,DL,14869,12478,1705,0.0,0.0,2336,-8.0,0.0,0.0
2,2013,4,19,5,DL,14057,14869,600,-4.0,0.0,851,-15.0,0.0,0.0
3,2013,4,19,5,DL,15016,11433,1630,28.0,1.0,1903,24.0,1.0,0.0
4,2013,4,19,5,DL,11193,12892,1615,-6.0,0.0,1805,-11.0,0.0,0.0


In [3]:
# ## Create a vector of 0 of length equal to the number of rows
# temp = [0] * df.shape[0]
# temp = np.zeros((df.shape[0],1))
# cnt_out = 0
# ## test each outlier condition and mark with a 1 as required
# for i, x in enumerate(df['ArrDelay']):
#         if (x > 100): 
#             temp[i] = 1
#             cnt_out += 1
    

# # # for i, x in enumerate(df['DepDelay']):
# # #     if (x > 100): temp[i] = 1 
# # # for i, x in enumerate(df['Carrier']):
# # #     if (x > 40): temp[i] = 1      
# df['Outlier'] = temp # append a column to the data frame
# df

In [4]:

# # for col in plot_cols: # loop over the columns
# fig = plt.figure(figsize=(6, 6))
# ax = fig.gca()
# ## Loop over the zip of the four vectors an subset the data and
# ## create the plot using the aesthetics provided
# # for o, f, c, m in zip(outlier, fuel, color, marker):
# for o in df['Outlier']:
#     temp = df['Outlier'].loc[(df['Outlier'] == o)] 
#     print(temp)
# #     if temp > 0:                    
# #         temp.plot(kind = 'scatter', x = df['ArrDelay'][o], y = df['DepDelay'][o] , 
# #                    ax = ax)                                 
# # ax.set_title('Scatter plot of lnprice vs. ' + col)

In [5]:
qnt = df['ArrDelay'].quantile([.25,.75])
print(qnt)

# IQR
# # First quartile (Q1) 
# Q1 = np.median(df['ArrDelay']) 
  
# # Third quartile (Q3) 
# Q3 = np.median(df['ArrDelay']) 

Q1 = df['ArrDelay'].quantile(0.25)
Q3 = df['ArrDelay'].quantile(0.75)
IQR = Q3 - Q1


upper = qnt.iloc[1] + (1.5*IQR)
lower = qnt.iloc[0] - (1.5*IQR)

print(Q1,Q3)
print(qnt.iloc[0],qnt.iloc[1])

df['ArrDelay'].shape[0]

0.25   -11.0
0.75    10.0
Name: ArrDelay, dtype: float64
-11.0 10.0
-11.0 10.0


2719418

In [6]:
df[df['ArrDelay'] > upper] = None
df[df['ArrDelay'] < lower] = None
# df.dropna()
new_data = df.dropna(axis = 0, how ='any')
# len(df)
# df['ArrDelay'].shape[0]

# dd = 2435411
# print(dd)
new_data.shape[0]
new_data

,Year,Month,DayofMonth,DayOfWeek,Carrier,OriginAirportID,DestAirportID,CRSDepTime,DepDelay,DepDel15,CRSArrTime,ArrDelay,ArrDel15,Cancelled
0,2013.0,4.0,19.0,5.0,DL,11433.0,13303.0,837.0,-3.0,0.0,1138.0,1.0,0.0,0.0
1,2013.0,4.0,19.0,5.0,DL,14869.0,12478.0,1705.0,0.0,0.0,2336.0,-8.0,0.0,0.0
2,2013.0,4.0,19.0,5.0,DL,14057.0,14869.0,600.0,-4.0,0.0,851.0,-15.0,0.0,0.0
3,2013.0,4.0,19.0,5.0,DL,15016.0,11433.0,1630.0,28.0,1.0,1903.0,24.0,1.0,0.0
4,2013.0,4.0,19.0,5.0,DL,11193.0,12892.0,1615.0,-6.0,0.0,1805.0,-11.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2719413,2013.0,10.0,19.0,6.0,DL,13204.0,14869.0,730.0,1.0,0.0,1013.0,-3.0,0.0,0.0
2719414,2013.0,10.0,19.0,6.0,DL,10397.0,13495.0,944.0,-2.0,0.0,1018.0,-3.0,0.0,0.0
2719415,2013.0,10.0,19.0,6.0,DL,10140.0,10397.0,1312.0,18.0,1.0,1807.0,9.0,0.0,0.0
2719416,2013.0,10.0,19.0,6.0,DL,10397.0,13244.0,1735.0,10.0,0.0,1757.0,7.0,0.0,0.0
